# $\Gamma$-Point Phonon (Dynamic Stabilities) and Formation Energies

In this notebook, we provides the interactive plots to explore the relationships between the lowest $\Gamma$-point phonon frequencies and the formation energies for free-standing perovskite membranes (similar to Fig. 4 in the manuscript).

In [18]:
try:
    from ase.db import connect
except ImportError:
    !pip install ase
    from ase.db import connect

try:
    import plotly 
except ImportError:
    !pip install plotly
import plotly.graph_objects as go

from utils import *
import pandas as pd
import numpy as np

In [3]:
#Step I - load all data into a panda dataframe
all_data_dict={}

database_counter=0
bulk_db=connect('2dpv_set_bulk.db')

for i in range(len(A_site_list)): 
    for orientation in ['100','110','111']:
        for term_type_id, term_type in enumerate(termination_types[orientation]): 
            this_db = connect('2dpv_set_'+orientation+'_'+term_type+'.db')
            
            for count_a, a in enumerate(A_site_list[i]):
                for b in B_site_list[i]:
                    for c in C_site_list[i]:
                        for thick in [3,5,7,9]:
                            system_name = a + b + c
                            uid = system_name + '3_' + str(orientation) + "_" + str(term_type) + "_" + str(thick)
                            
                            #getting the formation energy for the corresponding 2D free-standing perovskites
                            twod_formation_e = None
                            try:
                                row = this_db.get(selection=[('uid', '=', uid)])
                                twod_formation_e = row.key_value_pairs['formation_energy']
                            except KeyError:
                                print('WARNING: No value for ' + uid + ' Skip!')
                                continue
                            
                            if twod_formation_e is None:
                                continue
                                
                            #getting the lowest Gamma point phonon eigenfrequency for this 2D free-standing perovskite
                            # that is calculated from the Density Functional Perturbation Theory
                            min_gamma_freq = None
                            try:
                                gamma_point_freqs = row.data['gamma_phonon_freq']
                                try:
                                    gamma_point_freqs = [(f ** 2).real for f in gamma_point_freqs if f.imag != 0.0]
                                    min_gamma_freq = min(gamma_point_freqs)
                                except ValueError:
                                    # print("No imaginary frequency for " + str(uid))
                                    pass
                            except KeyError:
                                # print("Phonon calculation failed for " + str(uid))
                                pass
                                
                            #Organising the data into a dictionary that can be subsequently converted into
                            #a pandas dataframe
                            this_data_dict={'chemical_class':i+1,
                                            'system':uid.split("_")[0],
                                            'orientation':orientation,
                                            'termination':term_type,
                                            'thickness':thick,
                                            'energy':twod_formation_e,
                                            'gamma_point_freq':min_gamma_freq}
                            all_data_dict[database_counter]=this_data_dict
                            database_counter+=1
print(database_counter)                           

df = pd.DataFrame(all_data_dict).T

5088


In [13]:
chemical_classes = df['chemical_class'].unique().tolist()
crystallographic_orientations = df['orientation'].unique().tolist()
terminations = df['termination'].unique().tolist()
thicknesses = df['thickness'].unique().tolist()
#Step II - dataframe that is grouped by the chemical class, crystallographic orientations, terminations 
#layer thicknesses 
#We want to separate the plots for 2D perovskites with different thicknesess, as there are too many data in
#the plot, and there is not much significant thickness-dependent on the Gamma point phonon frequencies that
#can be observed anyway.
df_categorized = {}
for cc in chemical_classes:
    for cd in crystallographic_orientations:
        for term in terminations:
            for thickness in [3,5,7,9]:
                key = str(cc)+'_'+str(cd)+'_'+str(term)+'_'+str(thickness)
                this_set = df[(df['chemical_class']==cc) & (df['orientation']==cd) & (df['termination']==term) & (df['thickness']==thickness)]
            
                if not this_set.empty:
                    df_categorized[key] = this_set
                this_set=None

In [77]:
import copy

#Step III Start the plotting routine, where we want 
sorted_keys = list(sorted(df_categorized.keys()))
fig = go.Figure()

button_list=[]
display_this=[False for _ in sorted_keys]

for i,key in enumerate(sorted_keys):
    fig.add_trace(go.Scatter(x=df_categorized[key]['gamma_point_freq'],
                             y=df_categorized[key]['energy'],
                             #visible=True,
                             opacity=0.9,
                             marker=dict(size=12,color='#F49F05'),
                             marker_symbol='circle',#name=col,
                             text=df_categorized[key]['system'],
                             name=key
                            )
                 )
    _display_this = copy.deepcopy(display_this)
    _display_this[i] = True
    button_list.append(dict(label = key,
                            method = 'update',
                              args = [{'visible': _display_this},
                                      {'showlegend':False}]))



fig.update_layout(width=900, height=800,
    updatemenus=[go.layout.Updatemenu(
        active=0,
        buttons=button_list)])
    
fig.update_xaxes(title_text='$\\omega_{\\min}^{2}$')
fig.update_yaxes(title_text='$E_f^{2D}$')
fig.update_layout(yaxis_range=[-2.5,1.0],xaxis_range=[-6,1])
fig.update_traces(mode='markers', marker_line_width=2, marker_size=10)

fig.show()